# Day 17: エッジ検出（Sobel、Laplacian、Canny）

## Learning Objectives
- Sobel演算子によるエッジ検出を理解する
- Laplacianフィルタの原理と特性を把握する
- Cannyエッジ検出アルゴリズムを完全に理解する
- 各手法の特性と適用場面を学ぶ

---

# Part 1: Theory (2 hours)

## 17.1 エッジ検出の基本概念

エッジ検出は画像処理における最も基本的なタスクの一つで、物体の輪郭や境界を検出する技術でございます。

### 17.1.1 エッジとは？

```
【エッジの定義】

エッジは画像内の輝度値が急激に変化する領域でございます。

物理的な意味:
- 物体の境界
- 表面の変化（材質、照明）
- 深さや距離の不連続

数学的表現:
I(x,y) の勾配 ∇I = [∂I/∂x, ∂I/∂y] が大きい場所
勾配の大きさ: |∇I| = √[(∂I/∂x)² + (∂I/∂y)²]
勾配の方向: θ = tan⁻¹[(∂I/∂y)/(∂I/∂x)]
```

### 17.1.2 エッジ検出の評価指標

#### 1. 精度（Accuracy）

```
【評価方法】

- True Positive: 実際のエッジを正しく検出
- False Positive: 背景をエッジと誤検出
- False Negative: 実際のエッジを検出失敗

定量評価:
- Precision = TP / (TP + FP)
- Recall = TP / (TP + FN)
- F1-score = 2 × (Precision × Recall) / (Precision + Recall)
```

#### 2. 計算効率

```
【性能比較】

- Sobel: O(n) - 高速、リアルタイム処理に適す
- Laplacian: O(n) - 高速、二階微分なのでノイズに敏感
- Canny: O(n log n) - 遅いが高精度、最適化可能
```

## 17.2 Sobel演算子

### 17.2.1 Sobelの原理

```
【Sobelの特徴】

1. 一次微分による勾配計算
2. 3×3カーネルによる平滑化と微分の同時処理
3. 方向別に水平・垂直成分を検出

数学的表現:
G_x = Σ Σ I(x,y) × G_x'(i,j)
G_y = Σ Σ I(x,y) × G_y'(i,j)

勾配の大きさ: G = √(G_x² + G_y²)
勾配の方向: θ = tan⁻¹(G_y / G_x)
```

### 17.2.2 Sobelカーネル

```
【Sobelカーネルの設計】

水平方向（G_x）:
G_x = [-1  0  1]
      [-2  0  2]
      [-1  0  1]

垂直方向（G_y）:
G_y = [-1 -2 -1]
      [ 0  0  0]
      [ 1  2  1]

特徴:
- 中心の重みが大きい（微分強化）
- 周辺が小さい（平滑化効果）
- 45度方向のエッジも検出可能
```

### 17.2.3 Sobelの実装

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage

# Sobel演算子の実装
def sobel_edge_detection(image):
    """Sobelエッジ検出の実装"""
    # グレースケール変換
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Sobelカーネル
    sobel_x = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ], dtype=np.float32)
    
    sobel_y = np.array([
        [-1, -2, -1],
        [ 0,  0,  0],
        [ 1,  2,  1]
    ], dtype=np.float32)
    
    # 畳み込み演算
    grad_x = ndimage.convolve(image.astype(np.float32), sobel_x, mode='reflect')
    grad_y = ndimage.convolve(image.astype(np.float32), sobel_y, mode='reflect')
    
    # 勾配の大きさと方向
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    angle = np.arctan2(grad_y, grad_x) * 180 / np.pi
    
    return magnitude, angle, grad_x, grad_y

## 17.3 Laplacianフィルタ

### 17.3.1 Laplacianの原理

```
【Laplacianの特徴】

1. 二階微分によるエッジ検出
2. 方向に依存しない（ isotropic ）
3. エッジの両側を検出（zero-crossing）

数学的表現:
∇²I = ∂²I/∂x² + ∂²I/∂y²

離散化:
∇²I(x,y) = I(x+1,y) + I(x-1,y) + I(x,y+1) + I(x,y-1) - 4I(x,y)
```

### 17.3.2 Laplacianカーネル

#### 標準Laplacian

```
【4近傍Laplacian】
L = [ 0  1  0]
    [ 1 -4  1]
    [ 0  1  0]

【8近傍Laplacian】
L = [ 1  1  1]
    [ 1 -8  1]
    [ 1  1  1]

特徴:
- 中心が負、周辺が正
- エッジで値が大きくなる
- 方向に依存しない
```

#### LoG（Laplacian of Gaussian）

```
【LoGの利点】

1. ガウシアン平滑化でノイズ除去
2. 二階微分でエッジ検出
3. 「Marr-Hildreth演算子」とも呼ばれる

ガウシアン:
G(x,y) = (1/2πσ²) × exp(-(x²+y²)/2σ²)

LoG:
LoG = ∇²G = -(x²+y²-σ²)/(2πσ⁴) × exp(-(x²+y²)/2σ²)
```

### 17.3.3 Laplacianの実装

In [ ]:
# Laplacianフィルタの実装
def laplacian_edge_detection(image, use_log=False, sigma=1.0):
    """Laplacianエッジ検出の実装"""
    # グレースケール変換
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    if use_log:
        # LoG（Laplacian of Gaussian）
        from scipy.ndimage import gaussian_filter
        
        # ガウシアン平滑化
        smoothed = gaussian_filter(image.astype(np.float32), sigma=sigma)
        
        # Laplacian適用
        laplacian_kernel = np.array([
            [ 0,  1,  0],
            [ 1, -4,  1],
            [ 0,  1,  0]
        ], dtype=np.float32)
        
        edges = ndimage.convolve(smoothed, laplacian_kernel, mode='reflect')
    else:
        # 通常のLaplacian
        laplacian_kernel = np.array([
            [ 0,  1,  0],
            [ 1, -4,  1],
            [ 0,  1,  0]
        ], dtype=np.float32)
        
        edges = ndimage.convolve(image.astype(np.float32), laplacian_kernel, mode='reflect')
    
    # Zero-crossingの検出
    edges = np.abs(edges)
    
    return edges

## 17.4 Cannyエッジ検出アルゴリズム

### 17.4.1 Cannyの特徴

```
【Cannyの特徴】

1. 最適化されたエッジ検出アルゴリズム
2. 6段階の処理ステップ
3. 最大の SNR（信号対雑音比）を実現

設計基準:
- 優れた検出（エッジを漏らさない）
- 優れた定位（エッジの位置を正確に）
- 単一応答（1つのエッジに1つの応答）
```

### 17.4.2 Cannyの処理ステップ

#### ステップ1: ガウシアン平滑化

```
【ノイズ除去】

目的:
- 高周波ノイズの除去
- スムーズなエッジ検出

カーネルサイズの選択:
- σ = 1.0: 弱い平滑化
 - σ = 2.0: 中程度の平滑化
 - σ = 3.0: 強い平滑化
```

#### ステップ2: 勾配計算

```
【Sobel演算子の適用】

G_x = ∂I/∂x  （水平勾配）
G_y = ∂I/∂y  （垂直勾配）

勾配の大きさ: G = √(G_x² + G_y²)
勾配の方向: θ = tan⁻¹(G_y / G_x)

非极大値抑制の準備
```

#### ステップ3: 非极大値抑制

```
【エッジの幅を1ピクセルに】

アルゴリズム:
1. 勾配方向に隣接ピクセルを比較
2. もし現在のピクセルが最大なら保持
3. そうであれば、エッジではないとマーク

方向量子化:
- 0°, 45°, 90°, 135° の4方向に量子化
- 計算を簡略化
```

#### ステップ4: 閾値処理

```
【二値化】

低い閾値（low_threshold）
- 弱いエッジの候補
信頼性が低いがエッジを漏らさない

高い閾値（high_threshold）
- 強いエッジの候補
信頼性が高い

閾値比通常: 1:2 または 1:3
```

#### ステップ5: エッジトラッキング（ヒステリシス閾値）

```
【連続性の考慮】

アルゴリズム:
1. 強いエッジ（high_threshold以上）を開始点とする
2. 連続する弱いエッジを追跡
3. 弱いエッジがlow_threshold以上なら保持
4. low_thresholdを切ったら追跡終了

利点:
- エッジの連続性を維持
- 孤立したノイズを除去
```

### 17.4.3 Cannyの実装

In [ ]:
# Cannyエッジ検出の実装
def canny_edge_detection(image, low_threshold=0.05, high_threshold=0.15, sigma=1.0):
    """Cannyエッジ検出の実装"""
    # グレースケール変換
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # ステップ1: ガウシアン平滑化
    from scipy.ndimage import gaussian_filter
    smoothed = gaussian_filter(image.astype(np.float32), sigma=sigma)
    
    # ステップ2: 勾配計算
    sobel_x = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ], dtype=np.float32)
    
    sobel_y = np.array([
        [-1, -2, -1],
        [ 0,  0,  0],
        [ 1,  2,  1]
    ], dtype=np.float32)
    
    grad_x = ndimage.convolve(smoothed, sobel_x, mode='reflect')
    grad_y = ndimage.convolve(smoothed, sobel_y, mode='reflect')
    
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    # ステップ3: 非极大値抑制
    suppressed = non_max_suppression(magnitude, grad_x, grad_y)
    
    # ステップ4と5: ヒステリシス閾値処理
    edges = hysteresis_thresholding(suppressed, low_threshold, high_threshold)
    
    return edges.astype(np.uint8)

def non_max_suppression(magnitude, grad_x, grad_y):
    """非极大値抑制の実装"""
    # 方向の量子化
    angle = np.arctan2(grad_y, grad_x) * 180 / np.pi
    angle = np.where(angle < 0, angle + 180, angle)
    
    # 4方向に量子化
    angle = np.round(angle / 45) * 45 % 180
    
    # 出力配列
    suppressed = np.zeros_like(magnitude)
    
    # 各方向での非极大値抑制
    h, w = magnitude.shape
    
    for i in range(1, h-1):
        for j in range(1, w-1):
            current_mag = magnitude[i, j]
            current_angle = angle[i, j]
            
            if current_angle == 0:  # 水平方向
                neighbor1 = magnitude[i, j-1]
                neighbor2 = magnitude[i, j+1]
            elif current_angle == 45:  # 45度方向
                neighbor1 = magnitude[i-1, j+1]
                neighbor2 = magnitude[i+1, j-1]
            elif current_angle == 90:  # 垂直方向
                neighbor1 = magnitude[i-1, j]
                neighbor2 = magnitude[i+1, j]
            else:  # 135度方向
                neighbor1 = magnitude[i-1, j-1]
                neighbor2 = magnitude[i+1, j+1]
            
            # もし現在のピクセルが最大なら保持
            if current_mag >= neighbor1 and current_mag >= neighbor2:
                suppressed[i, j] = current_mag
    
    return suppressed

def hysteresis_thresholding(image, low_threshold, high_threshold):
    """ヒステリシス閾値処理の実装"""
    # 閾値を正規化された値に変換
    max_val = np.max(image)
    if max_val > 0:
        high_threshold = high_threshold * max_val
        low_threshold = low_threshold * max_val
    
    # 強いエッジと弱いエッジをマーク
    strong_edges = (image > high_threshold).astype(np.uint8)
    weak_edges = ((image > low_threshold) & (image <= high_threshold)).astype(np.uint8)
    
    # エッジトラッキング
    final_edges = np.zeros_like(image)
    
    # 強いエッジを起点に探索
    for i in range(1, image.shape[0]-1):
        for j in range(1, image.shape[1]-1):
            if strong_edges[i, j] == 1:
                # 強いエッジは保持
                final_edges[i, j] = 255
                # 8近傍の弱いエッジも保持
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        if weak_edges[i+di, j+dj] == 1:
                            final_edges[i+di, j+dj] = 255
    
    return final_edges

---

# Part 2: Practice (2 hours)

それでは、学んだ知識を実際に使ってみましょう！

## Exercise 17.1: Sobelエッジ検出の実装

In [ ]:
# テスト画像の読み込み
def create_test_image():
    """テスト用のシンプルな画像を作成"""
    image = np.zeros((200, 200), dtype=np.uint8)
    
    # 白い四角形
    image[50:150, 50:150] = 200
    
    # 斜めの線
    for i in range(200):
        image[i, i] = 200
    
    # 円
    center_x, center_y = 150, 100
    radius = 40
    for y in range(200):
        for x in range(200):
            if (x - center_x)**2 + (y - center_y)**2 <= radius**2:
                image[y, x] = 200
    
    return image

# テスト画像を作成
test_image = create_test_image()

# Sobelエッジ検出の実行
magnitude, angle, grad_x, grad_y = sobel_edge_detection(test_image)

# 結果の可視化
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.imshow(test_image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(grad_x, cmap='gray')
plt.title('Gradient X')
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(grad_y, cmap='gray')
plt.title('Gradient Y')
plt.axis('off')

plt.subplot(2, 3, 4)
plt.imshow(magnitude, cmap='gray')
plt.title('Gradient Magnitude')
plt.axis('off')

plt.subplot(2, 3, 5)
magnitude_normalized = magnitude / magnitude.max() * 255
plt.imshow(magnitude_normalized, cmap='gray')
plt.title('Normalized Magnitude')
plt.axis('off')

plt.subplot(2, 3, 6)
angle_normalized = (angle + 180) / 360 * 255
plt.imshow(angle_normalized, cmap='hsv')
plt.title('Gradient Direction')
plt.axis('off')

plt.tight_layout()
plt.show()

## Exercise 17.2: Laplacianエッジ検出の比較

In [ ]:
# Laplacianエッジ検出の比較
laplacian_edges = laplacian_edge_detection(test_image, use_log=False)
log_edges = laplacian_edge_detection(test_image, use_log=True, sigma=1.0)

# OpenCVのLaplacianとの比較
opencv_laplacian = cv2.Laplacian(test_image, cv2.CV_64F)
opencv_laplacian = np.abs(opencv_laplacian)

# 結果の可視化
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(laplacian_edges, cmap='gray')
plt.title('Laplacian Filter')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(log_edges, cmap='gray')
plt.title('LoG (σ=1.0)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(opencv_laplacian, cmap='gray')
plt.title('OpenCV Laplacian')
plt.axis('off')

plt.tight_layout()
plt.show()

# 異なるσのLoGの比較
plt.figure(figsize=(15, 5))

sigmas = [0.5, 1.0, 2.0]
for i, sigma in enumerate(sigmas):
    log_sigma = laplacian_edge_detection(test_image, use_log=True, sigma=sigma)
    plt.subplot(1, 3, i+1)
    plt.imshow(log_sigma, cmap='gray')
    plt.title(f'LoG (σ={sigma})')
    plt.axis('off')

plt.tight_layout()
plt.show()

## Exercise 17.3: Cannyエッジ検出のパラメータ調整

In [ ]:
# Cannyエッジ検出のパラメータ調整
def compare_canny_parameters(image, low_thresholds=[0.05, 0.1, 0.2], high_threshold_ratios=[2, 3, 4]):
    """Cannyのパラメータ比較"""
    plt.figure(figsize=(15, 10))
    
    idx = 1
    for low_thresh in low_thresholds:
        for ratio in high_threshold_ratios:
            high_thresh = low_thresh * ratio
            
            canny_edges = canny_edge_detection(
                image, 
                low_threshold=low_thresh,
                high_threshold=high_thresh,
                sigma=1.0
            )
            
            plt.subplot(3, 3, idx)
            plt.imshow(canny_edges, cmap='gray')
            plt.title(f'Low={low_thresh:.2f}, High={high_thresh:.2f}')
            plt.axis('off')
            
            idx += 1
    
    plt.suptitle('Canny Parameter Comparison', fontsize=16)
    plt.tight_layout()
    plt.show()

# パラメータ比較
compare_canny_parameters(test_image)

## Exercise 17.4: 実画像での比較

In [ ]:
# 実画像での比較
from google.colab.patches import cv2_imshow
import requests
from io import BytesIO

# 画像の読み込み
try:
    url = 'https://upload.wikimedia.org/wikipedia/commons/4/43/Lenna.png'
    response = requests.get(url)
    image = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
except:
    # エラー時はテスト画像を使用
    image = test_image

# 各手法の実行
sobel_edges = sobel_edge_detection(image)
laplacian_edges = laplacian_edge_detection(image, use_log=True, sigma=1.0)
canny_edges = canny_edge_detection(image, low_threshold=0.1, high_threshold=0.2, sigma=1.0)

# 結果の可視化
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.imshow(sobel_edges, cmap='gray')
plt.title('Sobel Edge Detection')
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(laplacian_edges, cmap='gray')
plt.title('LoG Edge Detection')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(canny_edges, cmap='gray')
plt.title('Canny Edge Detection')
plt.axis('off')

plt.tight_layout()
plt.show()

## Exercise 17.5: 性能比較と評価

In [ ]:
# 性能比較
import time

# 各手法の実行時間を計測
methods = {
    'Sobel': sobel_edge_detection,
    'Laplacian': lambda img: laplacian_edge_detection(img, use_log=True),
    'Canny': lambda img: canny_edge_detection(img, low_threshold=0.1, high_threshold=0.2)
}

# 結果の格納
results = {}
for name, method in methods.items():
    start_time = time.time()
    if name == 'Canny':
        result = method(image)
    else:
        result = method(image)
    end_time = time.time()
    
    results[name] = {
        'time': end_time - start_time,
        'edges': result
    }
    print(f"{name}: {end_time - start_time:.4f}秒")

# エッジ密度の比較
plt.figure(figsize=(12, 8))

bar_width = 0.25
index = np.arange(len(methods))

# 実行時間
times = [results[name]['time'] for name in methods.keys()]
plt.subplot(2, 1, 1)
plt.bar(index, times, bar_width, color='blue', alpha=0.7)
plt.xlabel('Method')
plt.ylabel('Execution Time (s)')
plt.title('Performance Comparison')
plt.xticks(index, methods.keys())

# エッジピクセル数
edge_counts = []
for name in methods.keys():
    edges = results[name]['edges']
    count = np.sum(edges > 0)
    edge_counts.append(count)

plt.subplot(2, 1, 2)
plt.bar(index, edge_counts, bar_width, color='green', alpha=0.7)
plt.xlabel('Method')
plt.ylabel('Edge Pixels')
plt.title('Edge Detection Results')
plt.xticks(index, methods.keys())

plt.tight_layout()
plt.show()

# 結果の要約
print("\n=== 性能比較結果 ===")
for name in methods.keys():
    print(f"{name}:")
    print(f"  - 実行時間: {results[name]['time']:.4f}秒")
    print(f"  - エッジピクセル数: {np.sum(results[name]['edges'] > 0)}")
    print()

---

# Self-Check (理解度確認)

本日の学習内容を確認しましょう：

## 基礎知識
- [ ] Sobel演算子の原理とカーネルを理解した
- [ ] Laplacianフィルタの特性とLoGの利点を理解した
- [ ] Cannyアルゴリズムの6段階を理解した

## 技術的知識
- [ ] 勾配計算と非极大値抑制の概念を理解した
- [ ] ヒステリシス閾値処理の利点を理解した
- [ ] 各手法の特徴と適用場面を理解した

## 実践力
- [ ] Sobelエッジ検出を実装した
- [ ] LaplacianとLoGの実装を比較した
- [ ] Cannyのパラメータ調整を行った
- [ ] 実際の画像で各手法を比較した

**お疲れ様でした！** Day 17はこれで終了です。

次回（Day 18）は「画像の幾何学変換」を学びます。